# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [31]:
import os
import requests
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display

In [32]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [3]:
# set up environment

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.split() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
elif api_key[:8] != "sk-proj-":
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
else :
    print("API key found and looks good so far!")

API key found and looks good so far!


In [5]:
openai=OpenAI()

In [24]:
system_prompt = """
You are a helpful programming tutor.
You will be given a coding question or snippet.
Your job is to explain what the code does and why it works in a way that is very easy for a beginner to understand.

Requirements:
1. Use simple, beginner-friendly language (avoid jargon unless you also explain it).
2. Break down the code step by step.
3. Explain both *what* the code does and *why* it is written that way.
4. Use small examples or analogies if it helps clarity.
5. Keep the explanation short, clear, and approachable.
6. Respond in Markdown format.
"""

In [25]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [26]:
def user_prompt_for(question):
    user_prompt = ""
    user_prompt += "You will be given a question.\n"
    user_prompt += "Your task is to explain the answer in a way that is simple and beginner-friendly.\n"
    user_prompt += "Always respond in Markdown format for clarity.\n\n"
    user_prompt += "Here is the question:\n"
    user_prompt += question
    return user_prompt

In [27]:
messages = [
    {'role':'user','content':user_prompt_for(question)},
    {'role':'system','content':system_prompt}
]

In [33]:
# Get gpt-4o-mini to answer, with streaming

def get_answer(question):
    stream = openai.chat.completions.create(
        model = MODEL_GPT,
        messages = messages,
        stream = True
    )
    response = " "
    display_handle = display(Markdown(""),display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace('```','').replace('markdown','')
        update_display(Markdown(response), display_id = display_handle.display_id)

In [34]:
get_answer(question)

 Sure! Let’s break down the code step-by-step to understand what it does and why it's written that way.

### Code Breakdown

python
yield from {book.get("author") for book in books if book.get("author")}


1. **`books`**: This is likely a collection, such as a list, of dictionaries where each dictionary represents a book. For example:

   python
   books = [
       {"title": "Book 1", "author": "Author A"},
       {"title": "Book 2", "author": "Author B"},
       {"title": "Book 3", "author": None},  # No author
       {"title": "Book 4", "author": "Author A"}
   ]
   

2. **List Comprehension**: The code inside the curly braces `{ ... }` is using a feature called a "set comprehension." This is similar to list comprehension, but it creates a set, which is a type of collection that only keeps unique items (no duplicates).

   - Here, `book.get("author") for book in books` means "for each `book` in the `books`, get the `author` of that book."

3. **Filtering with `if`**: The part `if book.get("author")` checks if the author exists (not `None`). If an author is present, that name is included in the set. If there's no author (like in Book 3), it won't be added.

4. **`yield from`**: This is a way to return values from a generator function. When this part runs, it gives back each author from the set one by one when needed, instead of all at once.

### What Does This Code Do?

When you put it all together, this code does the following:

- It goes through each book in a list called `books`.
- It retrieves the author's name for each book, but only if that name exists.
- It collects all the unique authors into a set (which automatically removes duplicates).
- It then provides a way to get each author one at a time when you ask for them, thanks to `yield from`.

### Why Is It Written This Way?

- **Efficiency**: By using a set, it automatically removes duplicate authors, so you only get distinct names.
- **Readability**: The use of comprehensions makes the code cleaner and more compact.
- **Lazy Evaluation**: Using `yield from` allows you to produce values only as needed, which can save memory if you're working with large data sets; you don’t hold everything in memory at once.

### Quick Example

If we take our example `books`, the output of this code would be:

python
{"Author A", "Author B"}


This means you have two distinct authors, "Author A" and "Author B". "Author A" only appears once, even though they wrote two books.

### Conclusion

In summary, this code is a smart, efficient way to get a unique list of authors from a collection of books while ensuring that you only consider books that have authors listed.

In [54]:
OLLAMA_API = "http://localhost:11434/api/chat"

In [55]:
messages_ollama= [
    {'role':'user','content':user_prompt_for(question)}
]

In [56]:
payload = {
    'messages' : messages_ollama,
    'stream' : False,
    'model' : MODEL_LLAMA
}

In [57]:
responses = requests.post(OLLAMA_API ,json=payload)

In [61]:
ollama_via_openai = OpenAI(base_url = "http://localhost:11434/v1", api_key="ollama")

In [64]:
# Get Llama 3.2 to answer

def get_answers_open_source(question):
    response = ollama_via_openai.chat.completions.create(
        messages = messages_ollama,
        model = MODEL_LLAMA
    )
    result = response.choices[0].message.content
    return result

In [67]:
print(display(Markdown(get_answers_open_source(question))))

**Understanding Generator Expressions**
=====================================

### Broken Down Explanation

The provided code snippet utilizes a feature called "generator expressions" or "list comprehensions with `yield from`".
```python
yield from {book.get("author") for book in books if book.get("author")}
```
Let's break it down:

*   **List Comprehension**: The `{...}` part is similar to a list comprehension in Python, but instead of creating an actual list, it yields values one by one.
*   `for book in books`: This loop goes through each item (book) in the `books` collection.
*   `if book.get("author")`: It filters out only those books that have an 'author' key present in their dictionary (`book`). If a book does not have this key, it is skipped.

**What the Code Does**
-------------------

The code generates all authors from the filtered list of books. 

Here's how:

*   When one book has an author, this value will be included.
*   When no such book exists for which its "author" is found (None), there won't be any values generated.

**Why it's Used**
----------------

Generator expressions with `yield from` provide the benefit of iterating through every item more efficiently than using regular for loops. 

They offer the following advantages:

1.  **Memory Efficiency**: Instead of loading and storing all items in memory at once, it loads just one value (and only that many). 
    This feature is very efficient with large data sets, especially those stored as files.
2.  **Lazy Evaluation**: It uses lazy evaluation to evaluate expressions that are computed and returned on demand by these generators.
3.  **Higher Memory Usage but Lower Memory Access Time**

*   Instead of loading the whole item list into memory at once
*   loads one element at a time, which is typically faster than accessing that many elements before

**When to Use it**
------------------

This type of code should be used whenever:

1.  *Efficiency Matters*: If you're dealing with massive data sets.
2.  *Storage Space is limited*: When every bit counts or space must be kept to the minimum required, 
    such as an embedded system environment.

*   There are many more scenarios in which this can come up

    - In real-time applications where resources might get occupied, the code here would be quite optimal.

In summary, Python's generator expression provides a powerful tool for dealing with large items without having to load every single one into memory at once.

None
